In [38]:
from bs4 import BeautifulSoup
import pandas as pd

###
### Load symbol data info from page "quote-header-info"
###
def soupGetSymbolInfo(s, data = None):
    if data is None:
        data = {}
    symbol_hdr_info = s.find("div", {"id":"quote-header-info"} )
    if symbol_hdr_info is None:
            return None
    symbol_name = symbol_hdr_info.find('fin-streamer', {'data-field':'marketState'})
    data['DATA_SYMBOL'] = symbol_name['data-symbol']
    symbol_time = symbol_hdr_info.find('div', {'id':'quote-market-notice'})
    #print(symbol_time)
    data['DATA_TIME'] = symbol_time.text
    return data

###
### Process the left and right table from quote page
###
def soupFindTable(s_table, data=None):
    if data is None:
        data = {} 
    tbody = s_table.find('tbody')
    rows = tbody.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        data[cols[0].text] = cols[1].text
    return data

###
### extract basic stock infomation from yahoo finance web site.
### given a stock
### return the basic finanical info in dictionary form.
###
def getStockInfo(stock):
    url_summary = 'https://finance.yahoo.com/quote/{0}?p={0}'
    ### hack the request to fool yahoo finance
    web_header={'Connection': 'keep-alive',
            'Expires': '-1',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) \
            AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.99 Safari/537.36'
           }
    response = requests.get(url_summary.format(stock), headers=web_header)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    ### setup return dict for symbol
    symbol_info = {}
    if soupGetSymbolInfo(soup, symbol_info) is None:
        print("MISSING SYMBOL {}".format(stock))
        return None
    if stock != symbol_info['DATA_SYMBOL']:
        print("SYMBOL {} != DATA_SYMBOL {}".format(stock, symbol_info['DATA_SYMBOL']) )
        print(symbol_info)
        return None

    script_data_left = soup.find("div", {"data-test":"left-summary-table"} )
    script_data_right = soup.find("div", {"data-test":"right-summary-table"} )
    
    soupFindTable(script_data_left, symbol_info)
    soupFindTable(script_data_right, symbol_info)
    
    return symbol_info
    
    

Testing the code here

In [40]:

df = pd.DataFrame ( [ getStockInfo(symbol) for symbol in "ENPH IBM RY".split() ] )
df


,DATA_SYMBOL,DATA_TIME,Previous Close,Open,Bid,Ask,Day's Range,52 Week Range,Volume,Avg. Volume,Market Cap,Beta (5Y Monthly),PE Ratio (TTM),EPS (TTM),Earnings Date,Forward Dividend & Yield,Ex-Dividend Date,1y Target Est
0,ENPH,At close: 04:00PM EDT,277.84,276.65,277.31 x 1000,278.00 x 1000,273.55 - 286.73,113.40 - 324.84,"2,865,894","3,595,722",37.585B,1.49,192.69,1.44,"Oct 24, 2022 - Oct 28, 2022",N/A (N/A),N/A,285.55
1,IBM,At close: 04:00PM EDT,121.63,121.66,118.90 x 800,118.75 x 800,118.61 - 122.43,114.56 - 144.73,"5,066,963","4,610,514",106.492B,0.84,19.51,6.09,"Oct 18, 2022",6.60 (5.56%),"Aug 09, 2022",142.68
2,RY,At close: 04:00PM EDT,90.84,91.10,89.21 x 800,92.92 x 1000,89.87 - 91.84,89.18 - 119.41,"901,344","804,130",126.565B,0.76,11.17,8.06,"Nov 29, 2022 - Dec 05, 2022",3.92 (4.36%),"Oct 25, 2022",104.94


Testing bad symbol data

In [41]:
getStockInfo('XYZ')

MISSING SYMBOL XYZ
